# Rback To the Future

In [2]:
import subprocess
import yaml
from typing import List
def get_checks(gh_actions_file: str):
    """Get checks from a GitHub Actions file"""
    with open(gh_actions_file) as f:
        actions = yaml.safe_load(f)
    steps = actions['jobs']['scavenger']['steps']
    checks = [step for step in steps if 'Checks' in step['name']]
    return checks

def run_check(check):
    # cmd = f"cd /workspaces/intro-to-kube\n{check['run']}"
    cmd = check['run']
    subprocess.run(
        cmd, 
        capture_output=False, 
        shell=True, 
        executable='/bin/bash',
        cwd = '/workspaces/intro-to-kube'
    )

def run_checks(checks: List):
    """Run checks"""
    for check in checks:
        run_check(check)
        
checks = get_checks('/workspaces/intro-to-kube/.github/workflows/rbac-to-the-future.yml')

## Setup

Let us start by firing up `minikube`, and running a script to provision `deployments` and `pods`.

In [3]:
!minikube start

😄  minikube v1.31.2 on Ubuntu 20.04 (docker/amd64)
✨  Automatically selected the docker driver. Other choices: none, ssh
📌  Using Docker driver with root privileges
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🔥  Creating docker container (CPUs=2, Memory=2200MB) ...
🐳  Preparing Kubernetes v1.27.4 on Docker 24.0.4 ...
    ▪ Generating certificates and keys ...
    ▪ Booting up control plane ...
    ▪ Configuring RBAC rules ...
🔗  Configuring bridge CNI (Container Networking Interface) ...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🔎  Verifying Kubernetes components...
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


In [4]:
%%bash
configs=("nginx-deployment.yml" "nginx-deployment.yml" "nginx-deployment.yml" "postgres-pod.yml" "postgres-pod.yml")
namespaces=("dev" "staging" "prod" "database" "backup")

for i in "${!configs[@]}"; do
    namespace="${namespaces[$i]}"
    kubectl create namespace $namespace
    kubectl config set-context --current --namespace=$namespace
    kubectl create -f "./k8s/${configs[$i]}"
done

namespace/dev created


Context "minikube" modified.
deployment.apps/nginx-deployment created
namespace/staging created
Context "minikube" modified.
deployment.apps/nginx-deployment created
namespace/prod created
Context "minikube" modified.
deployment.apps/nginx-deployment created
namespace/database created
Context "minikube" modified.
pod/postgres-pod created
namespace/backup created
Context "minikube" modified.
pod/postgres-pod created


In [5]:
!bash create_users.sh

Generating RSA private key, 2048 bit long modulus (2 primes)
.............................+++++
.....................................................................................+++++
e is 65537 (0x010001)
Signature ok
subject=CN = Jesus, O = Jesus
Getting CA Private Key
Generating RSA private key, 2048 bit long modulus (2 primes)
.......................................................................................................................+++++
.+++++
e is 65537 (0x010001)
Signature ok
subject=CN = Joey, O = Joey
Getting CA Private Key
Generating RSA private key, 2048 bit long modulus (2 primes)
.....................................................+++++
....+++++
e is 65537 (0x010001)
Signature ok
subject=CN = Jessica, O = Jessica
Getting CA Private Key
Generating RSA private key, 2048 bit long modulus (2 primes)
...........+++++
..........+++++
e is 65537 (0x010001)
Signature ok
subject=CN = Jules, O = Jules
Getting CA Private Key
User "Jesus" set.
Context "Jesus-context" 

In [6]:
!kubectl config view

apiVersion: v1
clusters:
- cluster:
    certificate-authority: /home/codespace/.minikube/ca.crt
    extensions:
    - extension:
        last-update: Tue, 10 Oct 2023 17:09:52 UTC
        provider: minikube.sigs.k8s.io
        version: v1.31.2
      name: cluster_info
    server: https://192.168.49.2:8443
  name: minikube
contexts:
- context:
    cluster: minikube
    user: Jessica
  name: Jessica-context
- context:
    cluster: minikube
    user: Jesus
  name: Jesus-context
- context:
    cluster: minikube
    user: Joey
  name: Joey-context
- context:
    cluster: minikube
    user: Jules
  name: Jules-context
- context:
    cluster: minikube
    extensions:
    - extension:
        last-update: Tue, 10 Oct 2023 17:09:52 UTC
        provider: minikube.sigs.k8s.io
        version: v1.31.2
      name: context_info
    namespace: backup
    user: minikube
  name: minikube
current-context: minikube
kind: Config
preferences: {}
users:
- name: Jessica
  user:
    client-certificate: /works

## Challenges

### Jules / Intern

> Let's start with easiest Role and RoleBinding to build. Let's give read-only access to the Intern role by defining get, watch, and list actions to Pod and Deployment resources strictly to the dev namespace. Afterwards bind this Role to Jules via the RoleBinding.

In [7]:
%%writefile roles/intern.yml
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: Intern
  namespace: dev
rules:
- apiGroups: [""] # "" indicates the core API group
  resources: ["pods", "deployments"]
  verbs: ["get", "watch", "list"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: InternBinding
  namespace: dev
subjects:
- kind: User
  name: Jules
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: Role
  name: Intern
  apiGroup: rbac.authorization.k8s.io

Overwriting roles/intern.yml


In [8]:
!kubectl apply -f roles/intern.yml

role.rbac.authorization.k8s.io/Intern created
rolebinding.rbac.authorization.k8s.io/InternBinding created


In [9]:
run_check(checks[0])

Switched to context "minikube".
role.rbac.authorization.k8s.io/Intern unchanged
rolebinding.rbac.authorization.k8s.io/InternBinding unchanged
Switched to context "Jules-context".
✅ Cannot access pods in non-dev namespace
✅ Cannot create/update resources in dev namespace
✅ Can access pods in dev namespace


### Jessica / DBAdmin

> The next Role to build will be DBAdmin. Here we want to grant the Database Admin the permissions they need to do any read/writes to any resources strictly within the database and backup namespaces. For this to happen, you will need to create a single Role, and then create multiple RoleBindings per namespaces for the user Jessica:

In [10]:
%%writefile roles/db-admin.yml
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: DBAdmin
  namespace: database
rules:
- apiGroups: [""]
  resources: ["pods", "deployments"]
  verbs: ["get", "watch", "list", "create", "update", "delete"]
---
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: DBAdminBinding
  namespace: database
subjects:
- kind: User
  name: Jessica
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: Role
  name: DBAdmin
  apiGroup: rbac.authorization.k8s.io
---
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: DBAdmin
  namespace: backup
rules:
- apiGroups: [""]
  resources: ["pods", "deployments"]
  verbs: ["get", "watch", "list", "create", "update", "delete"]
---
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: DBAdminBinding
  namespace: backup
subjects:
- kind: User
  name: Jessica
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: Role
  name: DBAdmin
  apiGroup: rbac.authorization.k8s.io

Overwriting roles/db-admin.yml


In [12]:
%%bash
kubectl config use-context minikube
kubectl apply -f roles/db-admin.yml

Switched to context "minikube".


role.rbac.authorization.k8s.io/DBAdmin created
rolebinding.rbac.authorization.k8s.io/DBAdminBinding created
role.rbac.authorization.k8s.io/DBAdmin created
rolebinding.rbac.authorization.k8s.io/DBAdminBinding created


In [13]:
run_check(checks[1])

Switched to context "minikube".
role.rbac.authorization.k8s.io/DBAdmin unchanged
rolebinding.rbac.authorization.k8s.io/DBAdminBinding unchanged
role.rbac.authorization.k8s.io/DBAdmin unchanged
rolebinding.rbac.authorization.k8s.io/DBAdminBinding unchanged
NAMESPACES
------------------------
database
backup
------------------------
role.rbac.authorization.k8s.io/DBAdmin unchanged
rolebinding.rbac.authorization.k8s.io/DBAdminBinding unchanged
role.rbac.authorization.k8s.io/DBAdmin unchanged
rolebinding.rbac.authorization.k8s.io/DBAdminBinding unchanged
Switched to context "Jessica-context".
✅ Can access pods in database namespace
✅ Can access pods in backup namespace
✅ Can create pods in database namespace
✅ Can create pods in backup namespace
✅ Can delete resources in database namespace
✅ Can delete resources in backup namespace
✅ Cannot get resources in namespaces other than database or backup
✅ Cannot create/update resources in namespaces other than database or backup


### Joey / Developer

In [15]:
%%writefile roles/developer.yml
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: Developer
  namespace: dev
rules:
- apiGroups: [""]
  resources: ["pods", "deployments"]
  verbs: ["get", "watch", "list", "create", "update"]
---
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: DeveloperBinding
  namespace: dev
subjects:
- kind: User
  name: Joey
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: Role
  name: Developer
  apiGroup: rbac.authorization.k8s.io
---
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: Developer
  namespace: staging
rules:
- apiGroups: [""]
  resources: ["pods", "deployments"]
  verbs: ["get", "watch", "list", "create", "update"]
---
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: DeveloperBinding
  namespace: staging
subjects:
- kind: User
  name: Joey
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: Role
  name: Developer
  apiGroup: rbac.authorization.k8s.io
---
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: Developer
  namespace: prod
rules:
- apiGroups: [""]
  resources: ["pods", "deployments"]
  verbs: ["get", "watch", "list", "create", "update"]
---
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: DeveloperBinding
  namespace: prod
subjects:
- kind: User
  name: Joey
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: Role
  name: Developer
  apiGroup: rbac.authorization.k8s.io

Overwriting roles/developer.yml


In [16]:
%%bash
kubectl config use-context minikube
kubectl apply -f roles/developer.yml

Switched to context "minikube".


role.rbac.authorization.k8s.io/Developer created
rolebinding.rbac.authorization.k8s.io/DeveloperBinding created
role.rbac.authorization.k8s.io/Developer created
rolebinding.rbac.authorization.k8s.io/DeveloperBinding created
role.rbac.authorization.k8s.io/Developer created
rolebinding.rbac.authorization.k8s.io/DeveloperBinding created


In [17]:
run_check(checks[2])

Switched to context "minikube".
role.rbac.authorization.k8s.io/Developer unchanged
rolebinding.rbac.authorization.k8s.io/DeveloperBinding unchanged
role.rbac.authorization.k8s.io/Developer unchanged
rolebinding.rbac.authorization.k8s.io/DeveloperBinding unchanged
role.rbac.authorization.k8s.io/Developer unchanged
rolebinding.rbac.authorization.k8s.io/DeveloperBinding unchanged
NAMESPACES
------------------------
dev
staging
prod
------------------------
role.rbac.authorization.k8s.io/Developer unchanged
rolebinding.rbac.authorization.k8s.io/DeveloperBinding unchanged
role.rbac.authorization.k8s.io/Developer unchanged
rolebinding.rbac.authorization.k8s.io/DeveloperBinding unchanged
role.rbac.authorization.k8s.io/Developer unchanged
rolebinding.rbac.authorization.k8s.io/DeveloperBinding unchanged
role.rbac.authorization.k8s.io/Developer unchanged
rolebinding.rbac.authorization.k8s.io/DeveloperBinding unchanged
role.rbac.authorization.k8s.io/Developer unchanged
rolebinding.rbac.authorizat

### Jesus / Kube Admin

>For our last role, we will work to build the KubeAdmin role. This role grants a lot more permissions, so instead of using the traditional Role and RoleBinding, we will define a ClusterRole and ClusterRoleBinding. This is perfect for non-namespaced resource management, which in our case, we hope that our KubeAdmin has all the necessary privileges in place to do any operation across all namespaces in our cluster. Grant this ClusterRoleBinding to Jesus.

In [18]:
%%writefile roles/kube-admin.yml
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: KubeAdmin
rules:
- apiGroups: [""]
  resources: ["pods", "deployments", "services", "configmaps", "secrets", "ingresses", "jobs", "cronjobs", "persistentvolumeclaims"]
  verbs: ["get", "watch", "list", "create", "update", "delete"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: KubeAdminClusterRoleBinding
subjects:
- kind: User
  name: Jesus
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: ClusterRole
  name: KubeAdmin
  apiGroup: rbac.authorization.k8s.io

Overwriting roles/kube-admin.yml


In [19]:
%%bash
kubectl config use-context minikube
kubectl apply -f roles/kube-admin.yml

Switched to context "minikube".
clusterrole.rbac.authorization.k8s.io/KubeAdmin created
clusterrolebinding.rbac.authorization.k8s.io/KubeAdminClusterRoleBinding created


In [20]:
run_check(checks[3])

Switched to context "minikube".
clusterrole.rbac.authorization.k8s.io/KubeAdmin unchanged
clusterrolebinding.rbac.authorization.k8s.io/KubeAdminClusterRoleBinding unchanged
Switched to context "Jesus-context".
✅ Can access pods in dev namespace
✅ Can access pods in staging namespace
✅ Can access pods in prod namespace
✅ Can access pods in database namespace
✅ Can access pods in backup namespace
✅ Can create pods in dev namespace
✅ Can create pods in staging namespace
✅ Can create pods in prod namespace
✅ Can create pods in database namespace
✅ Can create pods in backup namespace
✅ Can delete resources in dev namespace
✅ Can delete resources in staging namespace
✅ Can delete resources in prod namespace
✅ Can delete resources in database namespace
✅ Can delete resources in backup namespace


## Teardown

In [21]:
%%bash
echo "Deleting user_certs/*"
rm -rf user_certs
minikube stop
minikube delete

* Stopping node "minikube"  ...
* Powering off "minikube" via SSH ...
* 1 node stopped.
* Deleting "minikube" in docker ...
* Deleting container "minikube" ...
* Removing /home/codespace/.minikube/machines/minikube ...
* Removed all traces of the "minikube" cluster.
